
# Illustrates nested Loop Joins and lateral joins in SQL
  
cs3200: Database design  
https://northeastern-datalab.github.io/cs3200/  

CS 7240: Principles of scalable data management  
https://northeastern-datalab.github.io/cs7240/  

CS 7575: A Seminar On Relational Language Design    
https://northeastern-datalab.github.io/cs7575/  

SQL file 305  
https://github.com/northeastern-datalab/cs3200-activities/tree/master/sql    

First version: 1/26/2026  
This version: 1/29/2026  

### A. Nested Loop strategy with empty table (conceptual evaluation strategy)

In [1]:
print("--- 2nd nested loop ---")
for i in range(2):
    for j in range(3):
        for k in range(1):
            cond = "True" if (i == j or i == k) else ""
            print(f"i={i}, j={j}, k={k}: {cond}")            

--- 2nd nested loop ---
i=0, j=0, k=0: True
i=0, j=1, k=0: True
i=0, j=2, k=0: True
i=1, j=0, k=0: 
i=1, j=1, k=0: True
i=1, j=2, k=0: 


### B. Inner joins vs. lateral joins vs. nesed correlated queries

#### Input data

In [2]:
X = [1, 2, 3, 4]        # X = range(1, 5)
Y = [1, 2, 3, 4]

#### 1. Inner joins
Standard nested loop strategy for inner joins: the sequence of the loops does not matter

In [3]:
result = [(x, y) for x in X for y in Y if x < y]    # list comprehension: builds everything immediately

print(type(result))

for row in result:
    print(row)

<class 'list'>
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [4]:
result = ((x, y) for x in X for y in Y if x < y)    # generator expression: yields items lazily, one at a time

print(type(result))

for row in result:
    print(row)

<class 'generator'>
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [5]:
result = []
for x in X:
    for y in Y:
        if x < y:
            result.append((x, y))
            
for row in result:
    print(row)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [6]:
def generate_pairs(X, Y):           # as generator function that generates output pairs on demand
    for x in X:
        for y in Y:
            if x < y:
                yield (x, y)

for row in generate_pairs(X, Y):
    print(row)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [7]:
for x in X:
  for y in Y:
    cond = "True" if (x<y) else ""
    print(f"({x}, {y}): {cond}")

(1, 1): 
(1, 2): True
(1, 3): True
(1, 4): True
(2, 1): 
(2, 2): 
(2, 3): True
(2, 4): True
(3, 1): 
(3, 2): 
(3, 3): 
(3, 4): True
(4, 1): 
(4, 2): 
(4, 3): 
(4, 4): 


#### 2. Lateral join

In [8]:
result = [(x, z) for x in X for z in (y for y in Y if x < y)]

for row in result:
    print(row)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [9]:
for x in X:
  Z = [y for y in Y if x < y]
  for z in Z:
    cond = "True" if (x<y) else ""
    print(f"({x}, {z}): {cond}")

(1, 2): True
(1, 3): True
(1, 4): True
(2, 3): True
(2, 4): True
(3, 4): True


#### 3. Nested correlated query

In [10]:
result = [x for x in X if any(x < y for y in Y)]

for row in result:
    print(row)

1
2
3


In [11]:
result = [x for x in X if [y for y in Y if x < y]]

for row in result:
    print(row)

1
2
3


In [12]:
for x in X:
  cond = "True" if any(x < y for y in Y) else ""
  print(f"{x}: {cond}")

1: True
2: True
3: True
4: 


### 3. Lateral join example 

In [13]:
R = [
    ("a", 1),
    ("a", 2),
    ("b", 3),
    ("c", 4),
    ("c", 5),
    ("d", 6),
]

S = [
    ("a", 1),
    ("a", 2),
    ("b", 3),
    ("b", 4),
    ("c", 5),
    ("e", 6),
]

def lateral_X(rA):
    bs = [b for (sA, b) in S if sA < rA]
    sm = sum(bs) if bs else None   # SQL SUM(empty) = NULL
    return {"sm": sm}

result = [(rA, lateral_X(rA)["sm"]) for (rA, _) in R]
print(result)

[('a', None), ('a', None), ('b', 3), ('c', 10), ('c', 10), ('d', 15)]
